# Introduction

In this Juptyer Notebook file, we will introduce the _Particle Swarm Optimization_ technique, stored in `qittoolbox.algorithms.pso` . 

In [1]:
from qittoolbox.algorithms.pso import pso_sphere
from qittoolbox.algorithms.misc import von_neumann_entropy_gradient as gradient_helper
from qittoolbox.algorithms.misc.algorithm_helper_functions import get_complex_vector_from_coords_multi, get_density_matrix_from_vector_multi, get_von_neumann_entropy_multi

import qittoolbox.cqgchannels.SNpluschannel as channel
from qittoolbox.qfunctions.qfunctions import q0_bracket
from qittoolbox.linalg import basis_transformation


from math import sqrt
import numpy as np

from qittoolbox.logging.logger import set_loglevel
set_loglevel('info')

[info] : [set_loglevel] : set loglevel to `info`


## Case study: $N = 5$, $(k,l,m) = (2,1,1)$, $S_N^+$-channel $\hat{\Phi}_{k}^{(l),m}$

Let us investigate how the Particle Swarm Optimization technique is applied to an $S_N^+$-quantum channel:

In [2]:
N = 5
k = 2
l = 1
m = 1
trace_out_first = True
n_particles = 300
maxiter = 100

log_N = 10

channel_name = f'N{N}_k{k}_l{l}_m{m}_first{trace_out_first}'

pk = channel.get_Jones_Wenzl_projection_NC(k,N=N)
dim_pk = q0_bracket(2*k+1,N=sqrt(N))
rank_in = dim_pk
isom_in = basis_transformation.get_isometry_from_range_space_proj(pk, rank_in)

# Get the real dimensions...
n_dim = 2*dim_pk-1

#Get the channel description
kraus_ops = channel.get_kraus_ops(k,l,m,N=N,trace_out_first=trace_out_first)
kraus_ops_dense = [x.A for x in kraus_ops]
kraus_ops_H = [ x.getH().A for x in kraus_ops ]

isom_in_H = np.conj(isom_in.T)

def channel_function_single(rho_in: np.ndarray) -> np.ndarray:
    rho_in_larger = np.einsum('jk,kl->jl', np.einsum('ij,jl->il', isom_in, rho_in) , isom_in_H )
    return sum( np.einsum('jk,kl->jl', np.einsum('ij,jl->il',x,rho_in_larger), x_H) for x,x_H in zip(kraus_ops_dense,kraus_ops_H) )

def channel_function(rho_in: np.ndarray) -> np.ndarray:
    rho_in_larger = np.einsum('ijk,kl->ijl', np.einsum('ij,kjl->kil', isom_in, rho_in) , isom_in_H )
    return sum( np.einsum('ijk,kl->ijl', np.einsum('ij,kjl->kil',x,rho_in_larger), x_H) for x,x_H in zip(kraus_ops_dense,kraus_ops_H) )

def grad_channel_function(x: np.ndarray) -> np.ndarray:
    return gradient_helper.grad_vne_channel(channel_name, x, channel_function_single)   

def cost_channel_function(x: np.ndarray) -> np.ndarray:
    """(dim,n_particles) --> (n_particles,) cost function"""
    vectors = get_complex_vector_from_coords_multi(x)
    density_mats = get_density_matrix_from_vector_multi(vectors,make_copy=False)
    density_mats_out = channel_function(density_mats)
    return get_von_neumann_entropy_multi(density_mats_out)

In [3]:
output = pso_sphere.pso_sphere(cost_channel_function, n_dim, n_particles=n_particles, maxiter=maxiter, log_N=log_N)

[info] : [pso_sphere] PSO running with the following parameters: 
{'n_particles': 300, 'maxiter': 100, 'log_N': 10, 'nostalgia': 0.5, 'social': 0.1, 'inertial_weight': <function pso_sphere.<locals>.<lambda> at 0x000001D379050B80>, 'stopping_criterion': <function pso_sphere.<locals>.<lambda> at 0x000001D379050C10>, 'upper_bound_vel': 0.5, 'history_depth': 0}
[info] : [pso_sphere] After init, found best cost ct = 0.5334230246512015 located at index it = 256
[info] : [pso_sphere] Initialized, now starting the loop...
[info] : [pso_sphere]   % done        idx       weight                     ct         it         time
[info] : [pso_sphere]     10.0         10    1.000e+00     3.246748452099e-01        255          0.3
[info] : [pso_sphere]     20.0         20    1.000e+00     1.928110014215e-01        120          0.6
[info] : [pso_sphere]     30.0         30    1.000e+00     8.705562527193e-02        195          0.9
[info] : [pso_sphere]     40.0         40    1.000e+00     5.36204922041

The output variable `output` contains information about the current best position (`'gt'`), but also the current best cost (`'ct'`), and if the `history_depth` flag is set to a positive integer, it contains the last positions of all particles in `xt_hist`, up to a depth specified by `history_depth`.

In [4]:
output.keys()

dict_keys(['curr_idx', 'xt', 'vt', 'ft', 'pt', 'it', 'ct', 'gt', 'xt_hist', 'vt_hist', 'vt_proj_hist'])